In [5]:

import time
from typing import Any, Iterable

import gymnasium as gym
import numpy as np
import torch
import torch.distributions as dist
from gymnasium.vector import VectorEnv
from gymnasium.wrappers import RescaleAction, ClipAction
from torch import optim, nn

from src.datetime import get_current_timestamp
from src.model_db.tiny_model_db import TinyModelDB
from src.module_analysis import count_parameters
from src.moving_averages import ExponentialMovingAverage
from src.networks.core.seq_net import SeqNet
from src.networks.core.tensor_shape import TensorShape
from src.networks.core.torch_wrappers.torch_net import TorchNet
from src.np_functions import softmax
from src.reinforcement_learning.algorithms.policy_mitosis.async_policy_mitosis import AsyncPolicyMitosis
from src.reinforcement_learning.algorithms.ppo.ppo import PPO, PPOLoggingConfig
from src.reinforcement_learning.algorithms.policy_mitosis.policy_mitosis import PolicyMitosis, PolicyWithEnvAndInfo
from src.reinforcement_learning.core.callback import Callback
from src.reinforcement_learning.core.generalized_advantage_estimate import compute_gae_and_returns
from src.reinforcement_learning.core.normalization import NormalizationType
from src.reinforcement_learning.core.policies.actor_critic_policy import ActorCriticPolicy
from src.reinforcement_learning.core.policy_info import PolicyInfo
from src.reinforcement_learning.gym.envs.parallelize_env import parallelize_env_async
from src.reinforcement_learning.gym.envs.transform_reward_wrapper import TransformRewardWrapper
from src.stopwatch import Stopwatch
from src.summary_statistics import format_summary_statics
from src.torch_device import set_default_torch_device

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:



def init_policy():
    class A2CNetwork(nn.Module):

        def __init__(self):
            super().__init__()
            
            in_size = 376
            shared_out_sizes = []
            actor_out_sizes = [512, 512, 256, 256, 256, 256, 256, 256, 17]
            critic_out_sizes = [512, 512, 256, 256, 256, 1]
            
            hidden_activation_function = nn.ELU()
            actor_out_activation_function = nn.Tanh()
            critic_out_activation_function = nn.Identity()
            
            self.has_shared = len(shared_out_sizes) > 0
            
            if self.has_shared:
                self.shared = SeqNet.from_layer_provider(
                    layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
                        nn.Linear(in_features, out_features),
                        hidden_activation_function
                    ),
                    in_size=in_size,
                    out_sizes=shared_out_sizes
                )
            else:
                self.shared = TorchNet(nn.Identity(), in_shape=TensorShape(features=in_size), out_shape=TensorShape(features=in_size))

            self.actor = SeqNet.from_layer_provider(
                layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
                    nn.Linear(in_features, out_features),
                    actor_out_activation_function if is_last_layer else hidden_activation_function
                ),
                in_size=self.shared.out_shape.get_definite_features(),
                out_sizes=actor_out_sizes
            )

            self.critic = SeqNet.from_layer_provider(
                layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
                    nn.Linear(in_features, out_features),
                    critic_out_activation_function if is_last_layer else hidden_activation_function
                ),
                in_size=self.shared.out_shape.get_definite_features(),
                out_sizes=critic_out_sizes
            )

        def forward(self, x: torch.Tensor):
            if self.has_shared:
                shared_out = self.shared(x)
            else:
                shared_out = x

            return self.actor(shared_out), self.critic(shared_out)

    return ActorCriticPolicy(A2CNetwork(), lambda action_logits: dist.Normal(loc=action_logits, scale=policy_action_std))

def wrap_env(_env: VectorEnv):
    _env = TransformRewardWrapper(_env, lambda _reward: 0.01 * _reward) 
    _env = RescaleAction(_env, min_action=-1.0, max_action=1.0)
    _env = ClipAction(_env)
    return _env

def train_func(policy_with_env_and_info: PolicyWithEnvAndInfo) -> tuple[int, float]:
    policy = policy_with_env_and_info['policy']
    env = policy_with_env_and_info['env']
    
    score = 0.0
    score_ema = ExponentialMovingAverage(0.45)
    rollout_stopwatch = Stopwatch()
    def on_rollout_done(rl: PPO, step: int, info: dict[str, Any]):   
        
        if 'raw_rewards' in info['rollout']:
            raw_rewards = info['rollout']['raw_rewards']
            _, gamma_1_returns = compute_gae_and_returns(
                value_estimates=np.zeros_like(rl.buffer.rewards[:len(raw_rewards)]),
                rewards=raw_rewards,
                episode_starts=rl.buffer.episode_starts[:len(raw_rewards)],
                last_values=np.zeros_like(rl.buffer.rewards[0], dtype=float),
                last_dones=np.zeros_like(rl.buffer.episode_starts[0], dtype=bool),
                gamma=1.0,
                gae_lambda=1.0,
                normalize_rewards=None,
                normalize_advantages=None,
            )
        else:
            _, gamma_1_returns = rl.buffer.compute_gae_and_returns(
                last_values=torch.zeros_like(rl.buffer.value_estimates[0]),
                last_dones=np.zeros_like(rl.buffer.episode_starts[0], dtype=bool),
                gamma=1.0,
                gae_lambda=1.0,
                normalize_advantages=None,
                normalize_rewards=None,
            )
        
        episode_scores = gamma_1_returns[
            rl.buffer.episode_starts[:rl.buffer.pos]
        ]
        
        nonlocal score, score_ema
        score = episode_scores.mean()
        current_score_ema = score_ema.update(score)
        
        rollout_time = rollout_stopwatch.reset()
        
        resets = format_summary_statics(
            rl.buffer.episode_starts.astype(int).sum(axis=0), 
            mean_format='.2f',
            std_format=None,
            min_value_format='1d',
            max_value_format=None,
        )
        print(f'{step:>6}: '
              f'{score = :9.3f}, '
              f'score_ema = {current_score_ema:9.3f}, '
              f'time = {rollout_time:5.2f}, '
              f'{resets = :s}')
    
    policy_info_str = ('('
          f'policy_id = {policy_with_env_and_info["policy_id"]}, '
          f'parent_id = {policy_with_env_and_info["parent_policy_id"]}, '
          f'num_parameters = {count_parameters(policy)}, '
          f'previous_steps = {policy_with_env_and_info["policy_info"]["steps_trained"]}, '
          f'previous_score = {policy_with_env_and_info["policy_info"]["score"]:9.3f}'
          ')')
    
    print(f'Starting PPO with policy {policy_info_str:s} for {steps_per_iteration:_} steps')
    mitosis_iteration_stopwatch = Stopwatch()
    PPO(
        env=env,
        policy=policy.to(device),
        policy_optimizer=lambda pol: optim.Adam(pol.parameters(), lr=1e-5),
        buffer_size=2500,
        gamma=0.995,
        gae_lambda=1.0,
        normalize_rewards=None,
        normalize_advantages=NormalizationType.Std,
        weigh_actor_objective=lambda obj: 1.0 * obj,
        weigh_critic_objective=lambda obj: 0.5 * obj,
        ppo_max_epochs=10,
        ppo_kl_target=0.01,
        ppo_batch_size=500,
        action_ratio_clip_range=0.05,
        grad_norm_clip_value=2.0,
        callback=Callback(on_rollout_done=on_rollout_done),
        logging_config=PPOLoggingConfig(log_rollout_infos=True)
    ).train(steps_per_iteration)
    
    
    print(f'Training finished for policy {policy_info_str:s}, end score = {score:9.3f}, time = {mitosis_iteration_stopwatch.time_passed():6.2f}')
    
    return steps_per_iteration, score_ema.get()

def select_policy_selection_probs(policy_infos: Iterable[PolicyInfo]) -> np.ndarray:
    scores = np.array([policy_info['score'] for policy_info in policy_infos])
    scores = scores / scores.std()
    scores = softmax(scores, temperature=0.9)
    return scores

device = set_default_torch_device("cuda:0") if True else set_default_torch_device('cpu')

policy_action_std = 0.15
steps_per_iteration = 100_000

env_name = 'Humanoid-v4'
env_kwargs = {'forward_reward_weight': 1.25, 'healthy_reward': 0.5, 'ctrl_cost_weight': 0.001 }
num_envs = 32

# mitosis_id = get_current_timestamp()
mitosis_id = '2024-05-03_19.31.53'
policy_db = TinyModelDB[PolicyInfo](base_path=f'E:/saved_models/rl/{env_name}/mitosis-{mitosis_id}')

unwrapped_env = parallelize_env_async(lambda: gym.make(env_name, **env_kwargs), num_envs)
try:
    print(f'Starting mitosis with id {mitosis_id}')
    AsyncPolicyMitosis(
        num_workers=4,
        policy_db=policy_db,
        policy_train_function=train_func,
        env=unwrapped_env,
        new_init_policy_function=init_policy,
        new_wrap_env_function=wrap_env,
        select_policy_selection_probs=select_policy_selection_probs,
        min_base_ancestors=5,
        _globals=globals(),
        rng_seed=None,
    ).train_with_mitosis(1000)
except KeyboardInterrupt:
    print('keyboard interrupt')
finally:
    print('closing envs')
    time.sleep(2.5)
    unwrapped_env.close()
    print('envs closed')
    policy_db.close()
    print('model db closed')
    

print('done')

Starting mitosis with id 2024-05-03_19.31.53
Worker 0 starting iteration 1
Worker 1 starting iteration 2
Worker 2 starting iteration 3
Worker 3 starting iteration 4
policy selection probs = 
	2024-05-03_19.31.56~YOi2Kr: p = 0.003371, scores =  78.815, steps = 100000
	2024-05-03_19.41.26~ONGcgY: p = 0.002850, scores =  76.237, steps = 100000
	2024-05-03_19.50.38~rpp1TN: p = 0.003296, scores =  78.466, steps = 100000
	2024-05-03_19.59.45~iOUmuV: p = 0.001695, scores =  68.256, steps = 100000
	2024-05-03_20.08.31~qVIc0m: p = 0.003239, scores =  78.197, steps = 100000
	2024-05-03_20.23.19~jkSCqb: p = 0.004759, scores =  84.109, steps = 200000
	2024-05-03_20.32.46~UETzfs: p = 0.006956, scores =  89.935, steps = 300000
	2024-05-03_20.42.03~AAbsq9: p = 0.006533, scores =  88.973, steps = 300000
	2024-05-03_20.51.29~B2PjVU: p = 0.002693, scores =  75.363, steps = 100000
	2024-05-03_21.00.17~O81lZo: p = 0.004296, scores =  82.537, steps = 200000
	2024-05-03_21.09.57~l028kP: p = 0.010269, scores

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
def init_policy():
    import torch
    import torch.distributions as dist
    from torch import nn
    from src.networks.core.tensor_shape import TensorShape
    from src.networks.core.seq_net import SeqNet
    from src.networks.core.torch_wrappers.torch_net import TorchNet
    from src.reinforcement_learning.core.policies.actor_critic_policy import ActorCriticPolicy
    
    class A2CNetwork(nn.Module):

        def __init__(self):
            super().__init__()
            
            in_size = 376
            shared_out_sizes = []
            actor_out_sizes = [512, 512, 256, 256, 256, 256, 256, 256, 17]
            critic_out_sizes = [512, 512, 256, 256, 256, 1]
            
            hidden_activation_function = nn.ELU()
            actor_out_activation_function = nn.Tanh()
            critic_out_activation_function = nn.Identity()
            
            self.has_shared = len(shared_out_sizes) > 0
            
            if self.has_shared:
                self.shared = SeqNet.from_layer_provider(
                    layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
                        nn.Linear(in_features, out_features),
                        hidden_activation_function
                    ),
                    in_size=in_size,
                    out_sizes=shared_out_sizes
                )
            else:
                self.shared = TorchNet(nn.Identity(), in_shape=TensorShape(features=in_size), out_shape=TensorShape(features=in_size))

            self.actor = SeqNet.from_layer_provider(
                layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
                    nn.Linear(in_features, out_features),
                    actor_out_activation_function if is_last_layer else hidden_activation_function
                ),
                in_size=self.shared.out_shape.get_definite_features(),
                out_sizes=actor_out_sizes
            )

            self.critic = SeqNet.from_layer_provider(
                layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
                    nn.Linear(in_features, out_features),
                    critic_out_activation_function if is_last_layer else hidden_activation_function
                ),
                in_size=self.shared.out_shape.get_definite_features(),
                out_sizes=critic_out_sizes
            )

        def forward(self, x: torch.Tensor):
            if self.has_shared:
                shared_out = self.shared(x)
            else:
                shared_out = x

            return self.actor(shared_out), self.critic(shared_out)

    return ActorCriticPolicy(A2CNetwork(), lambda action_logits: dist.Normal(loc=action_logits, scale=policy_action_std))

init_policy()

ActorCriticPolicy(
  (network): A2CNetwork(
    (shared): TorchNet(
      (torch_module): Identity()
    )
    (actor): SeqNet(
      (layers): NetList(
        (0): TorchNet(
          (torch_module): Sequential(
            (0): Linear(in_features=376, out_features=512, bias=True)
            (1): ELU(alpha=1.0)
          )
        )
        (1): TorchNet(
          (torch_module): Sequential(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): ELU(alpha=1.0)
          )
        )
        (2): TorchNet(
          (torch_module): Sequential(
            (0): Linear(in_features=512, out_features=256, bias=True)
            (1): ELU(alpha=1.0)
          )
        )
        (3-7): 5 x TorchNet(
          (torch_module): Sequential(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): ELU(alpha=1.0)
          )
        )
        (8): TorchNet(
          (torch_module): Sequential(
            (0): Linear(in_features=256,

In [17]:
import cloudpickle
cloudpickle.dumps(init_policy)

b'\x80\x05\x95S\x12\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x94\x8c\x0e_make_function\x94\x93\x94(h\x00\x8c\r_builtin_type\x94\x93\x94\x8c\x08CodeType\x94\x85\x94R\x94(K\x00K\x00K\x00K\x02K\x07K\x03C\xbe\x87\x02\x87\x03\x87\x04\x87\x05\x87\x06\x87\x07\x97\x00d\x01d\x00l\x00\x8a\x07d\x01d\x00l\x01m\x02\x8a\x05\x01\x00d\x01d\x02l\x00m\x03\x8a\x06\x01\x00d\x01d\x03l\x04m\x05\x8a\x03\x01\x00d\x01d\x04l\x06m\x07\x8a\x02\x01\x00d\x01d\x05l\x08m\t\x8a\x04\x01\x00d\x01d\x06l\nm\x0b}\x00\x01\x00\x02\x00G\x00\x88\x02\x88\x03\x88\x04\x88\x06\x88\x07f\x05d\x07\x84\x08d\x08\x89\x06j\x0c\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x03\x00\x00\xab\x03\x00\x00\x00\x00\x00\x00\x00\x00}\x01\x02\x00|\x00\x02\x00|\x01\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x05f\x01d\t\x84\x08\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00S\x00\x94(NK\x00\x8c\x02nn\x94\x85\x94\x8c\x0bTensorShape\x94\x85\x94\x8c\x06SeqNet\x94\x85\x94\x8c\x08TorchNet\x94\x85\x94\x8c\x11ActorCriticPolicy\x

In [18]:
serialized = b'\x80\x05\x95S\x12\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x94\x8c\x0e_make_function\x94\x93\x94(h\x00\x8c\r_builtin_type\x94\x93\x94\x8c\x08CodeType\x94\x85\x94R\x94(K\x00K\x00K\x00K\x02K\x07K\x03C\xbe\x87\x02\x87\x03\x87\x04\x87\x05\x87\x06\x87\x07\x97\x00d\x01d\x00l\x00\x8a\x07d\x01d\x00l\x01m\x02\x8a\x05\x01\x00d\x01d\x02l\x00m\x03\x8a\x06\x01\x00d\x01d\x03l\x04m\x05\x8a\x03\x01\x00d\x01d\x04l\x06m\x07\x8a\x02\x01\x00d\x01d\x05l\x08m\t\x8a\x04\x01\x00d\x01d\x06l\nm\x0b}\x00\x01\x00\x02\x00G\x00\x88\x02\x88\x03\x88\x04\x88\x06\x88\x07f\x05d\x07\x84\x08d\x08\x89\x06j\x0c\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x03\x00\x00\xab\x03\x00\x00\x00\x00\x00\x00\x00\x00}\x01\x02\x00|\x00\x02\x00|\x01\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x05f\x01d\t\x84\x08\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00S\x00\x94(NK\x00\x8c\x02nn\x94\x85\x94\x8c\x0bTensorShape\x94\x85\x94\x8c\x06SeqNet\x94\x85\x94\x8c\x08TorchNet\x94\x85\x94\x8c\x11ActorCriticPolicy\x94\x85\x94h\x07(K\x00K\x00K\x00K\x00K\x05K\x00C>\x95\x05\x87\x00\x97\x00e\x00Z\x01d\x00Z\x02\x88\x01\x88\x02\x88\x03\x88\x00\x88\x04f\x05d\x01\x84\x08Z\x03d\x02\x94\x05j\x04\x00\x00\x00\x00\x00\x00\x00\x00f\x02d\x03\x84\x04Z\x05\x88\x00x\x01Z\x06S\x00\x94(\x8c\x1finit_policy.<locals>.A2CNetwork\x94h\x07(K\x01K\x00K\x00K\x05K\x07K\x13B\xca\x02\x00\x00\x95\x05\x87\x05\x87\x06\x87\x07\x97\x00t\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00d\x01}\x01g\x00}\x02g\x00d\x02\xa2\x01}\x03g\x00d\x03\xa2\x01}\x04\x89\x0c\xa0\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\x07\x89\x0c\xa0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\x05\x89\x0c\xa0\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\x06t\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00d\x04k\x04\x00\x00\x00\x00|\x00_\x06\x00\x00\x00\x00\x00\x00\x00\x00|\x00j\x06\x00\x00\x00\x00\x00\x00\x00\x00r"\x89\x08\xa0\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x07\x88\x0cf\x02d\x05\x84\x08|\x01|\x02\xac\x06\xa6\x03\x00\x00\xab\x03\x00\x00\x00\x00\x00\x00\x00\x00|\x00_\x08\x00\x00\x00\x00\x00\x00\x00\x00n9\x02\x00\x89\n\x89\x0c\xa0\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x89\t|\x01\xac\x07\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x89\t|\x01\xac\x07\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00\xac\x08\xa6\x03\x00\x00\xab\x03\x00\x00\x00\x00\x00\x00\x00\x00|\x00_\x08\x00\x00\x00\x00\x00\x00\x00\x00\x89\x08\xa0\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x05\x88\x07\x88\x0cf\x03d\t\x84\x08|\x00j\x08\x00\x00\x00\x00\x00\x00\x00\x00j\t\x00\x00\x00\x00\x00\x00\x00\x00\xa0\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x03\xac\x06\xa6\x03\x00\x00\xab\x03\x00\x00\x00\x00\x00\x00\x00\x00|\x00_\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x89\x08\xa0\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\x06\x88\x07\x88\x0cf\x03d\n\x84\x08|\x00j\x08\x00\x00\x00\x00\x00\x00\x00\x00j\t\x00\x00\x00\x00\x00\x00\x00\x00\xa0\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x04\xac\x06\xa6\x03\x00\x00\xab\x03\x00\x00\x00\x00\x00\x00\x00\x00|\x00_\x0c\x00\x00\x00\x00\x00\x00\x00\x00d\x00S\x00\x94(NMx\x01(M\x00\x02M\x00\x02M\x00\x01M\x00\x01M\x00\x01M\x00\x01M\x00\x01M\x00\x01K\x11t\x94(M\x00\x02M\x00\x02M\x00\x01M\x00\x01M\x00\x01K\x01t\x94K\x00h\x07(K\x04K\x00K\x00K\x04K\x06K\x13CX\x95\x02\x97\x00\x89\x05\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x89\x05\xa0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02|\x03\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00\x89\x04\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00S\x00\x94N\x85\x94\x8c\nSequential\x94\x8c\x06Linear\x94\x86\x94(\x8c\x08layer_nr\x94\x8c\ris_last_layer\x94\x8c\x0bin_features\x94\x8c\x0cout_features\x94t\x94\x8c?C:\\Users\\domin\\AppData\\Local\\Temp\\ipykernel_21372\\4056069569.py\x94\x8c\x08<lambda>\x94\x8c:init_policy.<locals>.A2CNetwork.__init__.<locals>.<lambda>\x94K\x1cC.\xf8\x80\x00\xd0^`\xd7^k\xd2^k\xd8\x18\x1a\x9f\t\x9a\t\xa0+\xa8|\xd1\x18<\xd4\x18<\xd8\x182\xf1\x05\x03_\x01\x16\xf4\x00\x03_\x01\x16\x80\x00\x94C\x00\x94\x8c\x1ahidden_activation_function\x94h\t\x86\x94)t\x94R\x94\x8c\x0elayer_provider\x94\x8c\x07in_size\x94\x8c\tout_sizes\x94\x87\x94\x8c\x08features\x94\x85\x94\x8c\x08in_shape\x94\x8c\tout_shape\x94\x86\x94h\x07(K\x04K\x00K\x00K\x04K\x06K\x13C`\x95\x03\x97\x00\x89\x06\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x89\x06\xa0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02|\x03\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00|\x01r\x02\x89\x04n\x01\x89\x05\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00S\x00\x94h\x19h\x1c(h\x1dh\x1eh\x1fh t\x94h"h#\x8c:init_policy.<locals>.A2CNetwork.__init__.<locals>.<lambda>\x94K\'C:\xf8\x80\x00\xd0Z\\\xd7Zg\xd2Zg\xd8\x14\x16\x97I\x92I\x98k\xa8<\xd1\x148\xd4\x148\xd85B\xd0\x14b\xd0\x141\xd0\x141\xd0Hb\xf1\x05\x03[\x01\x12\xf4\x00\x03[\x01\x12\x80\x00\x94h&\x8c\x1dactor_out_activation_function\x94h\'h\t\x87\x94)t\x94R\x94h\x07(K\x04K\x00K\x00K\x04K\x06K\x13C`\x95\x03\x97\x00\x89\x06\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x89\x06\xa0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02|\x03\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00|\x01r\x02\x89\x04n\x01\x89\x05\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00S\x00\x94h\x19h\x1c(h\x1dh\x1eh\x1fh t\x94h"h#\x8c:init_policy.<locals>.A2CNetwork.__init__.<locals>.<lambda>\x94K0C:\xf8\x80\x00\xd0Z\\\xd7Zg\xd2Zg\xd8\x14\x16\x97I\x92I\x98k\xa8<\xd1\x148\xd4\x148\xd86C\xd0\x14c\xd0\x142\xd0\x142\xd0Ic\xf1\x05\x03[\x01\x12\xf4\x00\x03[\x01\x12\x80\x00\x94h&\x8c\x1ecritic_out_activation_function\x94h\'h\t\x87\x94)t\x94R\x94t\x94(\x8c\x05super\x94\x8c\x08__init__\x94\x8c\x03ELU\x94\x8c\x04Tanh\x94\x8c\x08Identity\x94\x8c\x03len\x94\x8c\nhas_shared\x94\x8c\x13from_layer_provider\x94\x8c\x06shared\x94h2\x8c\x15get_definite_features\x94\x8c\x05actor\x94\x8c\x06critic\x94t\x94(\x8c\x04self\x94h,\x8c\x10shared_out_sizes\x94\x8c\x0factor_out_sizes\x94\x8c\x10critic_out_sizes\x94t\x94h"hF\x8c(init_policy.<locals>.A2CNetwork.__init__\x94K\x0cB\xcc\x01\x00\x00\xf8\xf8\xf8\xf8\x80\x00\xdd\x0c\x11\x89G\x8cG\xd7\x0c\x1c\xd2\x0c\x1c\xd1\x0c\x1e\xd4\x0c\x1e\xd0\x0c\x1e\xe0\x16\x19\x88G\xd8\x1f!\xd0\x0c\x1c\xd8\x1eJ\xd0\x1eJ\xd0\x1eJ\x88O\xd8\x1f;\xd0\x1f;\xd0\x1f;\xd0\x0c\x1c\xe0)+\xaf\x16\xaa\x16\xa9\x18\xac\x18\xd0\x0c&\xd8,.\xafG\xaaG\xa9I\xacI\xd0\x0c)\xd8-/\xaf[\xaa[\xa9]\xac]\xd0\x0c*\xe5\x1e!\xd0"2\xd1\x1e3\xd4\x1e3\xb0a\xd2\x1e7\x88D\x8cO\xe0\x0f\x13\x8c\x7f\xf0\x00\n\rG\x02\xd8\x1e$\xd7\x1e8\xd2\x1e8\xf0\x02\x03$\x16\xf0\x00\x03$\x16\xf0\x00\x03$\x16\xf0\x00\x03$\x16\xf0\x00\x03$\x16\xf0\x08\x00\x1d$\xd8\x1e.\xf0\r\x00\x1f9\xf1\x00\x07\x1f\x12\xf4\x00\x07\x1f\x12\x90\x04\x94\x0b\x90\x0b\xf0\x12\x00\x1f\'\x98h\xa0r\xa7{\xa2{\xa1}\xa4}\xb8{\xb8{\xd0T[\xd0?\\\xd1?\\\xd4?\\\xd0hs\xd0hs\xf0\x00\x00~\x01E\x02\xf0\x00\x00i\x01F\x02\xf1\x00\x00i\x01F\x02\xf4\x00\x00i\x01F\x02\xf0\x00\x00\x1fG\x02\xf1\x00\x00\x1fG\x02\xf4\x00\x00\x1fG\x02\x90\x04\x94\x0b\xe0\x19\x1f\xd7\x193\xd2\x193\xf0\x02\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x08\x00\x19\x1d\x9c\x0b\xd4\x18-\xd7\x18C\xd2\x18C\xd1\x18E\xd4\x18E\xd8\x1a)\xf0\r\x00\x1a4\xf1\x00\x07\x1a\x0e\xf4\x00\x07\x1a\x0e\x88D\x8cJ\xf0\x12\x00\x1b!\xd7\x1a4\xd2\x1a4\xf0\x02\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x00\x03 \x12\xf0\x08\x00\x19\x1d\x9c\x0b\xd4\x18-\xd7\x18C\xd2\x18C\xd1\x18E\xd4\x18E\xd8\x1a*\xf0\r\x00\x1b5\xf1\x00\x07\x1b\x0e\xf4\x00\x07\x1b\x0e\x88D\x8cK\x88K\x88K\x94h&(h\rh\x0bh\x0f\x8c\t__class__\x94h\tt\x94h8h@h\'\x87\x94t\x94R\x94\x8c\x01x\x94h\x07(K\x02K\x00K\x00K\x03K\x04K\x13C\x94\x97\x00|\x00j\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x16|\x00\xa0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x01\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00}\x02n\x02|\x01}\x02|\x00\xa0\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00|\x00\xa0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00f\x02S\x00\x94h\x19(hKhMhOhPt\x94hRh^\x8c\nshared_out\x94\x87\x94h"\x8c\x07forward\x94\x8c\'init_policy.<locals>.A2CNetwork.forward\x94K8CE\x80\x00\xd8\x0f\x13\x8c\x7f\xf0\x00\x03\r\x1f\xd8\x1d!\x9f[\x9a[\xa8\x11\x99^\x9c^\x90\n\x90\n\xe0\x1d\x1e\x90\n\xe0\x13\x17\x97:\x92:\x98j\xd1\x13)\xd4\x13)\xa84\xaf;\xaa;\xb0z\xd1+B\xd4+B\xd0\x13B\xd0\x0cB\x94h&))t\x94R\x94t\x94(\x8c\x08__name__\x94\x8c\n__module__\x94\x8c\x0c__qualname__\x94hF\x8c\x06Tensor\x94hc\x8c\r__classcell__\x94t\x94)h"\x8c\nA2CNetwork\x94h\x14K\nCt\xf8\xf8\x80\x00\x80\x00\x80\x00\x80\x00\x80\x00\xf0\x04*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0\x00*\t\x0e\xf0X\x01\x06\tC\x01\x98U\x9c\\\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\xf0\x00\x06\tC\x01\x94h&(h\rh\x0bh\x0fh\t\x8c\x05torch\x94t\x94hY\x85\x94t\x94R\x94hoh\x07(K\x01K\x00K\x00K\x01K\x04K\x13C<\x95\x01\x97\x00\x89\x01\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x00t\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xac\x01\xa6\x02\x00\x00\xab\x02\x00\x00\x00\x00\x00\x00\x00\x00S\x00\x94N\x8c\x03loc\x94\x8c\x05scale\x94\x86\x94\x86\x94\x8c\x06Normal\x94\x8c\x11policy_action_std\x94\x86\x94\x8c\raction_logits\x94\x85\x94h"h#\x8c\x1dinit_policy.<locals>.<lambda>\x94K@C\x19\xf8\x80\x00\xc0\x14\xc7\x1b\xc2\x1b\xd0Q^\xd5fw\xc0\x1b\xd1Ax\xd4Ax\x80\x00\x94h&\x8c\x04dist\x94\x85\x94)t\x94R\x94t\x94(hq\x8c\x13torch.distributions\x94\x8c\rdistributions\x94h\t\x8c\x1esrc.networks.core.tensor_shape\x94h\x0b\x8c\x19src.networks.core.seq_net\x94h\r\x8c*src.networks.core.torch_wrappers.torch_net\x94h\x0f\x8c<src.reinforcement_learning.core.policies.actor_critic_policy\x94h\x11\x8c\x06Module\x94t\x94h\x11ho\x86\x94h"\x8c\x0binit_policy\x94h\x90K\x01C\xfb\xf8\xf8\xf8\xf8\xf8\xf8\x80\x00\xd8\x04\x10\x80L\x80L\x80L\xd8\x04&\xd0\x04&\xd0\x04&\xd0\x04&\xd0\x04&\xd0\x04&\xd8\x04\x18\xd0\x04\x18\xd0\x04\x18\xd0\x04\x18\xd0\x04\x18\xd0\x04\x18\xd8\x04:\xd0\x04:\xd0\x04:\xd0\x04:\xd0\x04:\xd0\x04:\xd8\x040\xd0\x040\xd0\x040\xd0\x040\xd0\x040\xd0\x040\xd8\x04C\xd0\x04C\xd0\x04C\xd0\x04C\xd0\x04C\xd0\x04C\xd8\x04^\xd0\x04^\xd0\x04^\xd0\x04^\xd0\x04^\xd0\x04^\xf0\x044\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\xf0\x004\x05C\x01\x90R\x94Y\xf1\x004\x05C\x01\xf4\x004\x05C\x01\xf0\x004\x05C\x01\xf0l\x01\x00\x0c\x1d\xd0\x0b\x1c\x98Z\x98Z\x99\\\x9c\\\xd0+x\xd0+x\xd0+x\xd0+x\xd1\x0by\xd4\x0by\xd0\x04y\x94h&)(h\rh\x0bh\x0fh\x82h\thqt\x94t\x94R\x94}\x94(\x8c\x0b__package__\x94Nhi\x8c\x08__main__\x94uNNNt\x94R\x94h\x00\x8c\x12_function_setstate\x94\x93\x94h\x99}\x94}\x94(hih\x90hkh\x90\x8c\x0f__annotations__\x94}\x94\x8c\x0e__kwdefaults__\x94N\x8c\x0c__defaults__\x94Nhjh\x97\x8c\x07__doc__\x94N\x8c\x0b__closure__\x94N\x8c\x17_cloudpickle_submodules\x94]\x94\x8c\x0b__globals__\x94}\x94u\x86\x94\x86R0.'

import cloudpickle
import torch
cloudpickle.loads(serialized)().action_dist_provider(torch.Tensor([0]))

NameError: name 'policy_action_std' is not defined

In [ ]:
from stable_baselines3.common.distributions import DiagGaussianDistribution